# ***ASSIGNMENT - 18***

## Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

## **Task 1: Data Exploration and Preprocessing**

In [ ]:
# ------------------------------------------------------------
# 1: Data Loading and Exploration
# ------------------------------------------------------------

data = pd.read_csv("sonardataset.csv")

print("\nFirst 5 rows of the dataset:")
print(data.head())

print("\nDataset shape (rows, columns):")
print(data.shape)

print("\nDataset information:")
print(data.info())

print("\nClass distribution:")
print(data.iloc[:, -1].value_counts())

# ------------------------------------------------------------
# 2: Feature–Target Separation
# ------------------------------------------------------------

# SONAR dataset structure:
# 60 numerical features + 1 categorical target (M / R)

X = data.iloc[:, :-1]   # First 60 columns
y = data.iloc[:, -1]    # Last column (M or R)

# ------------------------------------------------------------
# 3: Encoding, Scaling, Train–Test Split
# ------------------------------------------------------------

# Encode target labels: M → 1, R → 0
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Train–test split (as per standard practice)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y)

# Feature scaling (very important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


First 5 rows of the dataset:
      x_1     x_2     x_3     x_4     x_5     x_6     x_7     x_8     x_9  \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

     x_10  ...    x_52    x_53    x_54    x_55    x_56    x_57    x_58  \
0  0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084   
1  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
2  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
3  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
4  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   

     x_59    x_60  Y  
0  0.0090  0.0032  R  
1  0.0052  0.004

## **Task 2: ANN Model Implementation**

In [ ]:
# ------------------------------------------------------------
# 1: Define ANN Architecture
# ------------------------------------------------------------

def build_model(neurons=16, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(neurons, activation="relu", input_shape=(60,)))
    model.add(Dense(neurons // 2, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"])
    return model

# ------------------------------------------------------------
# Task 2.2: Train Base ANN Model
# ------------------------------------------------------------

base_model = build_model(neurons=16, learning_rate=0.001)

base_model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    verbose=0)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## **Task 3: Hyperparameter Tuning**

In [ ]:
# ------------------------------------------------------------
# Task 3.1: Experiment with Different Hyperparameters
# ------------------------------------------------------------

configs = [
    {"neurons": 8, "lr": 0.001},
    {"neurons": 16, "lr": 0.001},
    {"neurons": 32, "lr": 0.001},
    {"neurons": 16, "lr": 0.01}]

best_accuracy = 0
best_model = None
best_config = None

for cfg in configs:
    model = build_model(neurons=cfg["neurons"], learning_rate=cfg["lr"])
    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

    preds = (model.predict(X_test) > 0.5).astype(int).flatten()
    acc = accuracy_score(y_test, preds)

    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model
        best_config = cfg

print("\nBest Hyperparameter Configuration:")
print(best_config)
print("Best Accuracy:", best_accuracy)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step

Best Hyperparameter Configuration:
{'neurons': 16, 'lr': 0.01}
Best Accuracy: 0.8809523809523809


## **Task 4: Evaluation**

In [ ]:
# ------------------------------------------------------------
# 1: Predictions Using Best Model
# ------------------------------------------------------------

y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()


# ------------------------------------------------------------
# 2: Evaluation Metrics
# ------------------------------------------------------------

print("\nFinal ANN Model Performance:")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1 Score :", f1_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

Final ANN Model Performance:
Accuracy : 0.8809523809523809
Precision: 0.9411764705882353
Recall   : 0.8
F1 Score : 0.8648648648648649

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.95      0.89        22
           1       0.94      0.80      0.86        20

    accuracy                           0.88        42
   macro avg       0.89      0.88      0.88        42
weighted avg       0.89      0.88      0.88        42



## **Theoretical Questions**
1. Why use Neural Networks for SONAR data?

- SONAR signals exhibit complex, non-linear patterns across frequency bands.
Neural Networks can model such non-linear relationships effectively.

2. What is Hyperparameter Tuning?

- Hyperparameter tuning involves selecting optimal values for parameters
such as number of neurons and learning rate to improve model performance.

3. Why Feature Scaling is Important?

- Scaling ensures all input features contribute equally during training
and prevents dominance of high-magnitude values.

4. Conclusion:
- The tuned ANN achieved better performance than the base model,
demonstrating the effectiveness of neural networks for
Mines vs Rocks classification.